In [8]:
# Model Architecture Params
input_seq_length = 8
output_seq_length = 12
seq_length = input_seq_length + output_seq_length
num_features = 2
hidden_dim = 128
embedding_dim = 64

# Discretization
neighborhood_size = 32
grid_size = 0.5

# Training Params
num_epochs = 1
batch_size = 5
learning_rate = 0.003

In [29]:
from dataloader import DataLoader

file_path = "../../data/eth/hotel/pixel_pos_interpolate.csv"
dataset_name = "eth_hotel"
file_path_processed = "../../data/preprocessed/"

dataloader = DataLoader(file_path, dataset_name, file_path_processed, force_process_data_flag=True)

Preprocess data
Write preprocessed data to ../../data/preprocessed/eth_hotel_data.pkl
Done preprocessing


In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, ReLU, LSTMCell
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop

import time

# Check TensorFlow version
tf_version = tf.__version__
print("TensorFlow version:", tf_version)

# Check if there is a GPU available
if tf.config.list_physical_devices('GPU'):
    print("GPU is available")
else:
    print("GPU is not available")

TensorFlow version: 2.15.0
GPU is available


2024-01-20 17:18:59.489886: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-20 17:18:59.675085: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-20 17:18:59.675132: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [5]:
import itertools

def get_occupancy_map(input, dense_representation, ped_id_to_index_map, num_peds_per_seq, ped_per_frame):
    grid_size_half = grid_size / 2.0
    # plot_animation(dense_representation, ped_id_to_index_map, [1,3])

    occupancy_grid = []

    for frame_id in range(seq_length):
        occupancy_grid_frame = np.zeros((num_peds_per_seq, num_peds_per_seq, neighborhood_size, neighborhood_size))
        
        ped_indices = [ped_id_to_index_map[ped_id] for ped_id in ped_per_frame[frame_id]]

        for ped_i, ped_j in itertools.permutations(ped_indices, 2):
            ped_i_y, ped_i_x = dense_representation[frame_id, ped_i]
            ped_j_y, ped_j_x = dense_representation[frame_id, ped_j]
            ped_i_neighbor_y_low, ped_i_neighbor_x_low = ped_i_y - grid_size_half, ped_i_x - grid_size_half
            ped_i_neighbor_y_high, ped_i_neighbor_x_high = ped_i_y + grid_size_half, ped_i_x + grid_size_half

            if ped_j_y >= ped_i_neighbor_y_low and ped_j_y <= ped_i_neighbor_y_high and ped_j_x >= ped_i_neighbor_x_low and ped_j_x <= ped_i_neighbor_x_high:
                # ped_j in ped_i neighborhood
                cell_y = int(np.floor((ped_j_y - ped_i_neighbor_y_low) / grid_size * neighborhood_size))
                cell_x = int(np.floor((ped_j_x - ped_i_neighbor_x_low) / grid_size * neighborhood_size))
                occupancy_grid_frame[ped_i, ped_j, cell_y, cell_x] = 1

        occupancy_grid.append(tf.Variable(tf.reshape(occupancy_grid_frame, (num_peds_per_seq, num_peds_per_seq, -1))))

    return occupancy_grid

In [27]:
class SocialLSTM(Model):
    def __init__(self, embedding_dim=64, hidden_dim=128, pool_size=(8,8), neighbors=32, num_features=2):
        super(SocialLSTM, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.input_embedding_layer = Dense(embedding_dim)
        self.social_embedding_layer = Dense(embedding_dim)  # TODO
        self.lstm = LSTMCell(hidden_dim)
        self.output_embedding_layer = Dense(num_features)
        
        # self.embedding_layer = Embedding(input_dim=num_features, output_dim=embedding_dim)
        # self.spatial_pooling = AveragePooling2D(pool_size=pool_size, strides=strides)

        self.relu = ReLU()


    def call(self, input, lstm_hidden, lstm_cell, occupancy_grid, ped_id_to_index_map, ped_per_frame):
        out_split = tf.split(tf.zeros(input.shape), num_or_size_splits=input.shape[0], axis=0)
        
        # if tf.config.list_physical_devices('GPU'):
        #     out_split = [tf.identity(x) for x in out_split]

            
        for frame_id in range(input.shape[0]):
            input_t = input[frame_id]
            input_embed_out = self.input_embedding_layer(input_t)
            input_embed_out = self.relu(input_embed_out)

            # TODO
            occupancy_grid_frame = occupancy_grid[frame_id]
            print(occupancy_grid_frame.shape, lstm_hidden.shape)

            # Option 1: matmul
            ped_indices = [ped_id_to_index_map[ped_id] for ped_id in ped_per_frame[frame_id]]
            for ped_idx in ped_indices:
                print("tmp shape:", occupancy_grid_frame[ped_idx].shape)  # Should be (10, 1024)
                print("tmp", occupancy_grid_frame[ped_idx])  # Should be (10, 1024)
                # print(occupancy_grid_frame[ped_idx])
                tmp = tf.transpose(lstm_hidden)
                print("lstm_hidden shape:", tmp.shape)    # Should be (128, 10)
                print("tmp:", tmp)
                out = tf.matmul(tmp, occupancy_grid_frame[ped_idx])     # Failed
                # print(out.shape)
                break
                
            # Option 2: tensor contraction
            # result = tf.einsum('ijk,jx->ikx', occupancy_grid_frame, lstm_hidden)
            # print(result.shape)

            social_embed_out = self.social_embedding_layer(input_t)
            social_embed_out = self.relu(social_embed_out)

            embed_out = tf.concat([input_embed_out, social_embed_out], axis=1)

            lstm_out, new_state = self.lstm(embed_out, (lstm_hidden, lstm_cell))
            lstm_hidden, lstm_cell = new_state
            self.lstm_hidden_memory = lstm_hidden
            

            out_t = self.output_embedding_layer(lstm_out)
            out_t = tf.expand_dims(out_t, axis=0)

            out_split[frame_id] = out_t
            break
        
        out = tf.concat(out_split, axis=0)
        return out

In [28]:
from utils import mse_loss

device = '/CPU:0'
if tf.config.list_physical_devices('GPU'):
    device = '/GPU:0'
    print("Using GPU...")

dataloader.reset_frame_ptr()
with tf.device(device):

    model = SocialLSTM()
    optimizer = RMSprop(learning_rate)

    for epoch in range(num_epochs):
        # For each batch  
        epoch_loss = 0.0      
        for batch in dataloader.generate_batches():
            start = time.time()

            inputs, targets = batch
            if not inputs or not targets:
                # Traverse to the end
                break
            ped_per_frame = dataloader.ped_indices

            batch_loss = 0.0
            
            # For each sequence
            for sequence_idx in range(batch_size):
                input, target = inputs[sequence_idx], targets[sequence_idx]
                dense_representation, ped_id_to_index_map = dataloader.convert_to_dense_representation(input) 
                dense_representation_tf = tf.Variable(tf.convert_to_tensor(dense_representation))
                num_peds_per_frame = len(ped_id_to_index_map)

                occupancy_grid = get_occupancy_map(input, dense_representation, ped_id_to_index_map, num_peds_per_frame, ped_per_frame)
                
                lstm_hidden = tf.Variable(tf.zeros((num_peds_per_frame, hidden_dim)))
                lstm_cell = tf.Variable(tf.zeros((num_peds_per_frame, hidden_dim)))


                with tf.GradientTape() as tape:
                    # Forward pass
                    out = model(dense_representation_tf, lstm_hidden, lstm_cell, occupancy_grid, ped_id_to_index_map, ped_per_frame)
                    loss = mse_loss(target, out, ped_id_to_index_map)
                    # loss = -tf.math.reduce_mean(out)    # TODO
                    batch_loss += loss


                # Compute gradients
                grads = tape.gradient(loss, model.trainable_variables)
                
                # Update parameters
                trainable_variables = model.trainable_variables
                optimizer.apply_gradients(zip(grads, trainable_variables))

                break
            break
        break

Using GPU...
(10, 10, 1024) (10, 128)
tmp shape: (10, 1024)
tmp tf.Tensor(
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]], shape=(10, 1024), dtype=float64)
lstm_hidden shape: (128, 10)
tmp: tf.Tensor(
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]], shape=(128, 10), dtype=float32)


InvalidArgumentError: Exception encountered when calling layer 'social_lstm_10' (type SocialLSTM).

cannot compute MatMul as input #1(zero-based) was expected to be a float tensor but is a double tensor [Op:MatMul] name: 

Call arguments received by layer 'social_lstm_10' (type SocialLSTM):
  • input=<tf.Variable 'Variable:0' shape=(20, 10, 2) dtype=float64, numpy=
array([[[ 0.25521 , -0.38722 ],
        [ 0.10903 , -0.57    ],
        [ 0.39896 , -0.19389 ],
        [ 0.47535 ,  0.11583 ],
        [-0.35069 ,  0.48611 ],
        [-0.37847 ,  0.53611 ],
        [-0.51389 ,  0.65278 ],
        [-0.30903 ,  0.26111 ],
        [ 0.2441  , -0.78361 ],
        [ 0.024306, -0.96083 ]],

       [[ 0.25347 , -0.39667 ],
        [ 0.11042 , -0.57889 ],
        [ 0.39167 , -0.18778 ],
        [ 0.475   ,  0.12611 ],
        [-0.35069 ,  0.48611 ],
        [-0.37847 ,  0.53611 ],
        [-0.51389 ,  0.65556 ],
        [-0.30903 ,  0.26111 ],
        [ 0.24514 , -0.79222 ],
        [ 0.024306, -0